# Multi-View-Majority-Vote-Learning-Algorithms-Direct-Minimization-of-PAC-Bayesian-Bounds

This Notebook contains everything necessary to reproduce the experiments in our paper:  

*Multi-View Majority Vote Learning Algorithms: Direct Minimization of PAC-Bayesian Bounds*

## Imports

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import OrderedDict

# Scikit-learn
from sklearn import preprocessing
from sklearn.utils import check_random_state
RAND = check_random_state(42)

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


from mvpb.dNDF import MultiViewBoundsDeepNeuralDecisionForests


# Import data
from mvlearn.datasets import load_UCImultifeature
from data.datasets import (SampleData,
                           MultipleFeatures,
                           Nutrimouse,
                           train_test_split,
                           train_test_merge,
                           s1_s2_split)
from mvpb.util import uniform_distribution

## Load and prepare the multiview datasets

In [2]:
dataset = MultipleFeatures()
X_train, y_train, X_test, y_test = dataset.get_data()
np.unique(y_train)
np.unique(y_test)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
Xs_train = []
Xs_test = []
for xtr, xts in zip(X_train, X_test):
    scaler = preprocessing.MinMaxScaler().fit(xtr)
    Xs_train.append(scaler.transform(xtr))
    Xs_test.append(scaler.transform(xts))

X_train_concat = np.concatenate(Xs_train, axis=1)
X_test_concat = np.concatenate(Xs_test, axis=1)

In [4]:
mvb = MultiViewBoundsDeepNeuralDecisionForests(nb_estimators=3,
                                               nb_views=len(Xs_train),
                                               depth =3,
                                               used_feature_rate=0.8,
                                               epochs=5)

In [5]:
mvb.fit(Xs_train,y_train)

View 1/6 done!
View 2/6 done!
View 3/6 done!
View 4/6 done!
View 5/6 done!
View 6/6 done!


MultiViewBoundsDeepNeuralDecisionForests(depth=3, epochs=5, nb_estimators=3,
                                         nb_views=6,
                                         posterior_rho=tensor([0.3333, 0.3333, 0.3333]),
                                         used_feature_rate=0.8)

In [6]:
posterior_Qv , posterior_rho = mvb.optimize_rho('Lambda')

Iteration: 0,	 Loss: 0.8961006989864078
Iteration: 1,	 Loss: 0.8960608571670065
Iteration: 2,	 Loss: 0.8959853018317202
Iteration: 3,	 Loss: 0.8958774344302098
Iteration: 4,	 Loss: 0.895740265622368
Iteration: 5,	 Loss: 0.8955769205178538
Iteration: 6,	 Loss: 0.8953895395837733
Iteration: 7,	 Loss: 0.8951807912925432
Iteration: 8,	 Loss: 0.8949523592266339
Iteration: 9,	 Loss: 0.8947062952444654
Iteration: 10,	 Loss: 0.8944440327962592
Iteration: 11,	 Loss: 0.8941669739760447
Iteration: 12,	 Loss: 0.8938767577845023
Iteration: 13,	 Loss: 0.8935743164328107
Iteration: 14,	 Loss: 0.8932606100244623
Iteration: 15,	 Loss: 0.8929366956108795
Iteration: 16,	 Loss: 0.8926034534803482
Iteration: 17,	 Loss: 0.8922616645469511
Iteration: 18,	 Loss: 0.891911861380063
Iteration: 19,	 Loss: 0.8915547515776944
Iteration: 20,	 Loss: 0.8911908229173781
Iteration: 21,	 Loss: 0.8908205422586715
Iteration: 22,	 Loss: 0.8904445984565436
Iteration: 23,	 Loss: 0.8900630440853579
Iteration: 24,	 Loss: 0.8896

In [7]:
mvb.mv_tandem_risk()

emp_tnd_rv=[1.0326651926065336, 1.078167413452801, 1.0312196001968343]


IndexError: tuple index out of range

In [7]:
_, mv_risk = mvb.predict_MV(Xs_test,y_test)
mv_risk

0.16000000000000003

In [8]:
lamb_bound, lamb_views = mvb.bound(bound="Lambda")

self.posterior_rho=tensor([0.1537, 0.1496, 0.1604, 0.2218, 0.1695, 0.1449]),  prior_pi=tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667])
KL_rhopi=tensor(0.0107),  KL_QP=tensor(0.0013)
len(emp_rv)=6
0.48270838101348457 tensor(0.0107)


In [9]:
lamb_bound, lamb_views

(1.0, [1.0, 1.0, 1.0, 0.8932343921151265, 1.0, 1.0])

In [10]:
mvb.bound(bound="PBkl")

self.posterior_rho=tensor([0.1537, 0.1496, 0.1604, 0.2218, 0.1695, 0.1449]),  prior_pi=tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667])
KL_rhopi=tensor(0.0107),  KL_QP=tensor(0.0013)
len(emp_rv)=6
empirical_gibbs_risk=0.48270838101348457, right_hand_side=tensor(0.0137)


(1.1303020506965373,
 [1.2293559737839155,
  1.2510526512864808,
  1.171988637760185,
  0.8333695602588967,
  1.1176942348792616,
  1.3065048439789029])

In [11]:
print(posterior_Qv[0])
print(posterior_Qv[1])
print(posterior_Qv[2])
print(posterior_Qv[3])
print(posterior_Qv[4])
print(posterior_Qv[5])

tensor([0.3539, 0.3318, 0.3143])
tensor([0.3519, 0.2919, 0.3562])
tensor([0.3207, 0.3116, 0.3678])
tensor([0.3326, 0.3327, 0.3347])
tensor([0.3377, 0.3328, 0.3295])
tensor([0.3230, 0.3430, 0.3340])


In [12]:
posterior_rho

tensor([0.1537, 0.1496, 0.1604, 0.2218, 0.1695, 0.1449])